In [1]:
import mikeio
import pandas as pd
import numpy as np
import os


# Function for adding Rainfall stations
# and also creating the rainfall stations weightage (influence) list for a catchment
def rf_weighted_ts_section(s_df, station_dict, starting_index = 0):
    
    # Getting Rainfall station name keys from the station dictionary and creating a list
    station_list = list(station_dict.keys())

    # Defining empty list for catchment rainfall timeseries file name
    # their corresponding influence on that catchment
    weighted_avg_ts_list = []
    station_weightage_list = []

    # Iterating through each stations and updating it in the stations list and weightage list
    for n, station in enumerate(station_list):
        
        rs_weightage = s_df.loc[:, station].values[0]
        
        param_file_name = '|.\\{}\\{}.dfs0|'.format(rainfall_folder_path_list[-1], station)
                                                        
        param_st_dict = {'station_no': starting_index + n,
                        'Filename': param_file_name,
                        'TimeSeriesItems': {'Item': [station_dict[station], 1]}}
        
        
        weighted_avg_ts_list.append(param_st_dict)
        station_weightage_list.append(rs_weightage)
    
    return weighted_avg_ts_list, station_weightage_list


# Function for adding Evaporations stations
# and also creating the Evaporations stations weightage (influence) list for a catchment
def evap_weighted_ts_section(s_df, evap_station_dict, start_index):
    
    # Getting Evaporation stations name keys from the station dictionary and creating a list
    evap_station_list = list(evap_station_dict.keys())

    # Defining empty list for catchment Evaporation timeseries file name
    # their corresponding influence on that catchment
    evap_weighted_avg_ts_list = []
    evap_station_weightage_list = []

    # Iterating through each stations and updating it in the stations list and weightage list
    for n, station in enumerate(evap_station_list):
        rs_weightage = (s_df.loc[:, station].values)[0]
        
        param_file_name = '|.\\{}\\{}.dfs0|'.format(evap_folder_path_list[-1], station)

        param_st_dict = {'station_no': start_index + n,
                        'Filename': param_file_name,
                        'TimeSeriesItems': {'Item': [evap_station_dict[station], 1]}}
        
        
        evap_weighted_avg_ts_list.append(param_st_dict)
        evap_station_weightage_list.append(rs_weightage)
    
    return evap_weighted_avg_ts_list, evap_station_weightage_list


# Function for creating the section for calculation of Mean Aeral Average for a catchment for the parameter
def parameter_ts_section_dict(catchment_name, datatype, ts_path, weighted_average_timeseries_list, weight_list):
    
    parameter_ts_dict = {'Catchment_Name': catchment_name,
                          'MAWCalculation': True,
                          'DataType': int(datatype),
                          'Timeseries': ts_path,
                          'TimeSeriesItems': {'Item': ['Weighted', 1]},
                          'Weighted_average_timeseries': {'Input_TimeSeries' : weighted_average_timeseries_list},
                          'Weighted_average_combinations': {'WeightCombination': {'Weight' : weight_list}},
                          'Temporal_distribution_timeseries': None,
                          'Temporal_distribution_combinations': None}
                        
    return parameter_ts_dict


# Reading mike rainfall-runoff parameter file template
pfs = mikeio.read_pfs(r"E:\Script\RR_Setup_File_Preparation\Input\Input_RR.rr11")

# Reading the file containing catchment area
catch_df = pd.read_excel(r"E:\Script\RR_Setup_File_Preparation\Input\Catchment_Area.xlsx")

# Reading the file that contains the parameter name and correstion file name with the column name in the file
rf_evap_df = pd.read_excel(r"E:\Script\RR_Setup_File_Preparation\Input\Rainfall_and_Evaporation_Item_list.xlsx")

# Reading the files containing the Rainfall and Evaporation station weightage for each catchment
rf_station_weight_df = pd.read_excel(r"E:\Script\RR_Setup_File_Preparation\Input\RF_Station_Weightage.xlsx")
evap_station_weight_df = pd.read_excel(r"E:\Script\RR_Setup_File_Preparation\Input\Evap_Station_Weightage.xlsx")

# Setting Rainfall data (timeseries) file path
rainfall_data_folder = r"E:\Script\RR_Setup_File_Preparation\Output\RF"
rainfall_folder_path_list = rainfall_data_folder.split(os.sep)

# Setting Evaporation data (timeseries) file path
evap_data_folder = r"E:\Script\RR_Setup_File_Preparation\Output\Evap"
evap_folder_path_list = evap_data_folder.split(os.sep)

# Setting the output directory
output_folder_path = r"E:\Script\RR_Setup_File_Preparation\Output"


# Iterating through all the item and their corresponding stations
# and storing it in a dictionary
station_name_dict = {}

for item in rf_evap_df['Item_Type'].unique():
    
    dfs0_df = rf_evap_df[rf_evap_df['Item_Type'] == item]
    
    station_dfs0_dict = {}
    
    for i in range(0, len(dfs0_df), 1):

        dfs0_Name = dfs0_df.iloc[i, 1]
        dfs0_Column_Name = dfs0_df.iloc[i, 2]

        station_dfs0_dict[dfs0_Name] = dfs0_Column_Name
    
    station_name_dict[item] = station_dfs0_dict


# Creating a 'Mean Areal(MA)' average output folder directory for each parameter
for p in station_name_dict.keys():
    ma_folder_path = os.path.join(output_folder_path, "MA{}".format(p[0].upper()))

    try:
        os.mkdir(ma_folder_path)
    
    except:
        pass


# Iterating through each catchment and updating it in the rainfall-runoff(RR) parameter file
for i, catchment in enumerate(catch_df['Catchement ID']):
    
    n_df = catch_df[catch_df['Catchement ID'] == catchment].values
    
    catchment_name = n_df[0][0]
    catchment_area = n_df[0][1]
    
    if i < 2:
        pfs.MIKE_RR.CatchList.Catchment[i].Catchment_Name = catchment_name
        pfs.MIKE_RR.CatchList.Catchment[i].Catchment_Area = "{:.2f}".format(catchment_area)
    
    else:
        pfs.MIKE_RR.CatchList.Catchment.append(pfs.MIKE_RR.CatchList.Catchment[0].copy())
        
        pfs.MIKE_RR.CatchList.Catchment[i].Catchment_Name = catchment_name
        pfs.MIKE_RR.CatchList.Catchment[i].Catchment_Area = "{:.2f}".format(catchment_area)
        
        pfs.MIKE_RR.ParameterList.NAM_Parameters.append(pfs.MIKE_RR.ParameterList.NAM_Parameters[0].copy())
        
        

# For each catchment creating the each parameter "Mean Aeral" average calculation section
# and setting output directory using the defined function
ts_section_list = []

    
for catch in rf_station_weight_df['Catchement ID']:

    rf_df = rf_station_weight_df[rf_station_weight_df['Catchement ID'] == catch]
    evap_df = evap_station_weight_df[evap_station_weight_df['Catchement ID'] == catch]
    

    item_num =0
    
    
    for m_para in station_name_dict.keys(): 
        dfs0_dict = station_name_dict[m_para]
        
        ma_ts_path = '|.\\{}\\{}_{}.dfs0|'.format("MA{}".format(m_para[0].upper()), catch, m_para)
    
        # RAINFALL SECTION
        # (rainfall item starts at '0' for a catchment appends all rainfall stations serially)
        if item_num == 0:
            rf_dict = dfs0_dict
            rf_weighted_average_timeseries_list, rf_weight_list = rf_weighted_ts_section(rf_df, dfs0_dict)
            rf_ts_section = parameter_ts_section_dict(catchment_name = catch, 
                                                      datatype = 0, 
                                                      ts_path = ma_ts_path, 
                                                      weighted_average_timeseries_list = rf_weighted_average_timeseries_list, 
                                                      weight_list = rf_weight_list)
            ts_section_list.append(rf_ts_section)
            item_num += len(rf_dict)

        # EVAPORATION SECTION
        # (rainfall item starts after the rainfll stations ends (rainfall start index + number of rainfall stations) 
        # for a catchment appends all rainfall stations serially)
        elif item_num == len(rf_dict) :
            evap_weighted_average_timeseries_list, evap_weight_list = evap_weighted_ts_section(evap_df, 
                                                                                               dfs0_dict,
                                                                                               item_num)
            evap_ts_section = parameter_ts_section_dict(catchment_name = catch, 
                                                        datatype = 1, 
                                                        ts_path = ma_ts_path, 
                                                        weighted_average_timeseries_list =evap_weighted_average_timeseries_list, 
                                                        weight_list = evap_weight_list)

            ts_section_list.append(evap_ts_section)
            item_num += len(dfs0_dict)

        else:
            pass

# Updating the created parameter section in the 'RR' file
pfs.MIKE_RR.TimeseriesList['Condition'] = ts_section_list

# Writing the rainfall-runoff parameter file in the output directory
pfs.write(os.path.join(output_folder_path, 'Rainfall_Runoff_Parameter.rr11'))

print('Finished')

Finished
